#### SQLite port_lite database: orders table
#### csv files: orders, ORDERS-PLACED

In [1]:
import calendar
import pandas as pd
import panel as pn
import seaborn as sns
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import init_notebook_mode, show
init_notebook_mode(all_interactive=False)

pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

def categorise(row):  
    if row['trade'] == 'B':
        return 'Buy'
    else:
        return 'Sell'
    
today = date.today()
today

<IPython.core.display.Javascript object>

datetime.date(2022, 3, 3)

### Begin of Tables in the process

In [2]:
cols = 'trade name qty price active reason market'.split()
colt = 'trans name qty target active spd current change percent reason market'.split()

In [3]:
format_dict = {
    'qty':'{:,}',
    'price':'{:.2f}','target':'{:.2f}','current':'{:.2f}','change':'{:.2f}','diff':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}'
}

In [4]:
pd.read_sql_query('SELECT * FROM orders ORDER BY id DESC LIMIT 1', conlite).style.format(format_dict)

,id,trade,name,qty,price,active,reason,market
0,29,B,TQM,"4,500",43.00,2,6L,SET100


In [5]:
tmp = pd.read_csv('../data/orders-log.csv')
tmp['trans'] = tmp.apply(lambda row: categorise(row), axis=1)
tmp['amount'] = tmp.qty * tmp.target
tmp[tmp.active == 2].style.background_gradient(cmap = 'Blues').format(format_dict)

,trade,name,spd,reason,market,qty,target,current,change,percent,active,trans,amount
3,B,HREIT,-2,DOS,SET,"10,000",8.95,9.05,0.10,+1.12%,2,Buy,"89,500.00"
4,B,KCE,-1,RD20pct,SET50,"1,000",58.00,58.25,-2.50,-4.12%,2,Buy,"58,000.00"
5,B,MAKRO,-4,ROUND,SET,"1,500",42.00,43.00,-0.25,-0.58%,2,Buy,"63,000.00"
6,B,MEGA,-2,6L,SET100,"1,500",43.00,43.50,-1.25,-2.79%,2,Buy,"64,500.00"
8,B,TMT,-5,ROUND,SET,"15,000",10.50,11.00,0.00,0.00%,2,Buy,"157,500.00"
9,B,TPIPL,-5,nan,SET,"60,000",1.70,1.75,0.01,+0.57%,2,Buy,"102,000.00"
10,B,TQM,-3,6L,SET100,"4,500",43.00,43.75,-2.75,-5.91%,2,Buy,"193,500.00"
13,S,CPNCG,2,DOS,SET,"20,000",12.50,12.30,0.00,0.00%,2,Sell,"250,000.00"
16,S,JASIF,0,DOS,SET,"10,000",11.20,11.20,-0.10,-0.88%,2,Sell,"112,000.00"
21,S,SINGER,6,20pct,SET100,"1,500",50.75,49.25,-0.25,-0.51%,2,Sell,"76,125.00"


In [6]:
tmp.head().sort_values('spd',ascending=False).style.hide_index().format(format_dict)

trade,name,spd,reason,market,qty,target,current,change,percent,active,trans,amount
B,KCE,-1,RD20pct,SET50,"1,000",58.00,58.25,-2.50,-4.12%,2,Buy,"58,000.00"
B,HREIT,-2,DOS,SET,"10,000",8.95,9.05,0.10,+1.12%,2,Buy,"89,500.00"
B,GLOBAL,-5,nan,SET50,"9,000",19.40,19.90,-0.10,-0.50%,1,Buy,"174,600.00"
B,ASP,-13,3L,SET,"30,000",3.60,3.86,0.00,0.00%,0,Buy,"108,000.00"
B,BAM,-13,nan,SET100,"10,000",20.50,21.80,0.00,0.00%,0,Buy,"205,000.00"


### End of Tables in the process

### Print to verify before upload file

In [67]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active
FROM orders
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)

df_tab = pn.widgets.Tabulator(orders, layout='fit_data', width=800)
df_tab

Tabulator(layout='fit_data', value=   trade    name  ..., width=800)

In [68]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)
orders[cols].to_csv(one_file, header=True, index=False)

### End of transactional process

In [69]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active
FROM orders
WHERE active = 2
ORDER BY trade, name'''
df_act_2 = pd.read_sql(sql, conlite)
df_act_2.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active
0,B,MAKRO,"1,500",42.00,"63,000.00",ROUND,SET,2
1,B,MEGA,"1,500",43.00,"64,500.00",6L,SET100,2
2,B,STA,"2,500",25.75,"64,375.00",RD25pct,SET50,2
3,B,TPIPL,"60,000",1.70,"102,000.00",,SET,2
4,S,CPNCG,"20,000",12.50,"250,000.00",DOS,SET,2
5,S,JASIF,"10,000",11.20,"112,000.00",DOS,SET,2
6,S,MEGA,"1,500",47.25,"70,875.00",10pct,SET100,2
7,S,PTT,"3,000",41.25,"123,750.00",5pct,SET50,2


In [70]:
orders_grp = df_act_2.groupby(['trade'])
orders_grp['amount'].sum()

trade
B    293875.0
S    556625.0
Name: amount, dtype: float64

In [71]:
mask = df_act_2.trade == 'B'
buy_stocks = df_act_2[mask]
buy_stocks.style.background_gradient(cmap = 'Blues').format(format_dict)

,trade,name,qty,price,amount,reason,market,active
0,B,MAKRO,"1,500",42.00,"63,000.00",ROUND,SET,2
1,B,MEGA,"1,500",43.00,"64,500.00",6L,SET100,2
2,B,STA,"2,500",25.75,"64,375.00",RD25pct,SET50,2
3,B,TPIPL,"60,000",1.70,"102,000.00",,SET,2


In [72]:
cash = 1_092_000
buy_amt = buy_stocks.amount.sum()
balance = cash - buy_amt 
cash, buy_amt, balance

(1092000, 293875.0, 798125.0)

In [73]:
mask = df_act_2.trade == 'S'
sell_stocks = df_act_2[mask]
sell_stocks.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active
4,S,CPNCG,"20,000",12.50,"250,000.00",DOS,SET,2
5,S,JASIF,"10,000",11.20,"112,000.00",DOS,SET,2
6,S,MEGA,"1,500",47.25,"70,875.00",10pct,SET100,2
7,S,PTT,"3,000",41.25,"123,750.00",5pct,SET50,2


In [74]:
sell_amt = round(sell_stocks.amount.sum(),2)
balance = cash + sell_amt 
(cash, sell_amt, balance)

(1092000, 556625.0, 1648625.0)

### After call ord-log (must call everytime that orders change)

In [75]:
df = pd.read_csv('../data/orders-log.csv')
df['trans'] = df.apply(lambda row: categorise(row), axis=1)
df['amount'] = df.qty * df.target

In [76]:
show(df[['trans','name','spd','reason','qty','target','amount','current','active']])

trans,name,spd,reason,qty,target,amount,current,active


In [77]:
mb = (df.trans == 'Buy') & (df.spd >= -5)
df[mb].sort_values(['name'],ascending=[True]).style.format(format_dict)

,trade,name,spd,reason,market,qty,target,current,change,percent,active,trans,amount
3,B,MAKRO,-3,ROUND,SET,"1,500",42.00,42.75,-0.25,-0.58%,2,Buy,"63,000.00"
6,B,STA,-2,RD25pct,SET50,"2,500",25.75,26.25,-0.25,-0.94%,2,Buy,"64,375.00"
8,B,TPIPL,-4,nan,SET,"60,000",1.70,1.74,-0.01,-0.57%,2,Buy,"102,000.00"


In [78]:
show(df[['trans','name','spd','reason','qty','target','current','percent','active']])

trans,name,spd,reason,qty,target,current,percent,active


In [79]:
ms = (df.trans == 'Sell') & (df.spd <= 6)
df[ms].sort_values(['name'],ascending=[True]).style.format(format_dict)

,trade,name,spd,reason,market,qty,target,current,change,percent,active,trans,amount
12,S,CPNCG,1,DOS,SET,"20,000",12.50,12.40,0.10,+0.81%,2,Sell,"250,000.00"
15,S,JASIF,1,DOS,SET,"10,000",11.20,11.10,-0.10,-0.89%,2,Sell,"112,000.00"
18,S,PTT,6,5pct,SET50,"3,000",41.25,39.75,0.00,0.00%,2,Sell,"123,750.00"


In [80]:
df[mb | ms].sort_values(['trade','name'],ascending=[True,True])

,trade,name,spd,reason,market,qty,target,current,change,percent,active,trans,amount
3,B,MAKRO,-3,ROUND,SET,1500,42.00,42.75,-0.25,-0.58%,2,Buy,63000.0
6,B,STA,-2,RD25pct,SET50,2500,25.75,26.25,-0.25,-0.94%,2,Buy,64375.0
8,B,TPIPL,-4,NaN,SET,60000,1.70,1.74,-0.01,-0.57%,2,Buy,102000.0
12,S,CPNCG,1,DOS,SET,20000,12.50,12.40,0.10,+0.81%,2,Sell,250000.0
15,S,JASIF,1,DOS,SET,10000,11.20,11.10,-0.10,-0.89%,2,Sell,112000.0
18,S,PTT,6,5pct,SET50,3000,41.25,39.75,0.00,0.00%,2,Sell,123750.0


In [81]:
df[mb | ms].shape[0]

6

In [82]:
mask = (df.trade == 'B')
df[mb | ms].nlargest(5, "amount")[["name","trans","amount"]].style.format(format_dict)

,name,trans,amount
12,CPNCG,Sell,"250,000.00"
18,PTT,Sell,"123,750.00"
15,JASIF,Sell,"112,000.00"
8,TPIPL,Buy,"102,000.00"
6,STA,Buy,"64,375.00"


### Select source of orders between these two

In [83]:
df_out = df[mb | ms][colt]
df_out.style.format(format_dict)

,trans,name,qty,target,active,spd,current,change,percent,reason,market
3,Buy,MAKRO,"1,500",42.00,2,-3,42.75,-0.25,-0.58%,ROUND,SET
6,Buy,STA,"2,500",25.75,2,-2,26.25,-0.25,-0.94%,RD25pct,SET50
8,Buy,TPIPL,"60,000",1.70,2,-4,1.74,-0.01,-0.57%,nan,SET
12,Sell,CPNCG,"20,000",12.50,2,1,12.40,0.10,+0.81%,DOS,SET
15,Sell,JASIF,"10,000",11.20,2,1,11.10,-0.10,-0.89%,DOS,SET
18,Sell,PTT,"3,000",41.25,2,6,39.75,0.00,0.00%,5pct,SET50


In [24]:
df_out = df[df.active == 2][colt]
df_out.style.format(format_dict)

,trans,name,qty,target,active,spd,current,change,percent,reason,market
3,Buy,HREIT,"10,000",8.95,2,-1,9.00,-0.05,-0.55%,DOS,SET
4,Buy,MAKRO,"1,500",42.00,2,-3,42.75,-0.25,-0.58%,ROUND,SET
5,Buy,MEGA,"1,500",43.00,2,-6,44.50,1.00,+2.30%,6L,SET100
7,Buy,TMT,"15,000",10.50,2,-5,11.00,0.00,0.00%,ROUND,SET
8,Buy,TPIPL,"60,000",1.70,2,-4,1.74,-0.01,-0.57%,nan,SET
9,Buy,TQM,"4,500",43.00,2,-7,44.75,1.00,+2.29%,6L,SET100
12,Sell,CPNCG,"20,000",12.50,2,2,12.30,0.00,0.00%,DOS,SET
15,Sell,JASIF,"10,000",11.20,2,2,11.00,-0.20,-1.79%,DOS,SET
20,Sell,SINGER,"1,500",50.75,2,11,48.00,-1.25,-2.54%,20pct,SET100
22,Sell,WHART,"10,000",12.80,2,5,12.30,-0.10,-0.81%,DOS,SET


In [84]:
file_name = 'ORDERS-PLACED.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(box_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(one_file, header=True, index=False)

### End of Output ORDERS-PLACED

In [85]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active
FROM orders
WHERE ACTIVE = 2
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)
orders.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active
0,B,MAKRO,"1,500",42.00,"63,000.00",ROUND,SET,2
1,B,MEGA,"1,500",43.00,"64,500.00",6L,SET100,2
2,B,STA,"2,500",25.75,"64,375.00",RD25pct,SET50,2
3,B,TPIPL,"60,000",1.70,"102,000.00",,SET,2
4,S,CPNCG,"20,000",12.50,"250,000.00",DOS,SET,2
5,S,JASIF,"10,000",11.20,"112,000.00",DOS,SET,2
6,S,MEGA,"1,500",47.25,"70,875.00",10pct,SET100,2
7,S,PTT,"3,000",41.25,"123,750.00",5pct,SET50,2


### Evening Process

In [86]:
new_active = 1
old_active = 2

In [87]:
sql = """
UPDATE orders 
SET active = %s
WHERE active = %s
"""
sql = sql % (new_active, old_active)
rp = conlite.execute(sql)
rp.rowcount

8

In [88]:
names = df_out['name']
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

"'MAKRO', 'STA', 'TPIPL', 'CPNCG', 'JASIF', 'PTT'"

In [89]:
sql = """
UPDATE orders 
SET active = 2
WHERE name IN (%s)
"""
sql = sql % (in_p)
rp = conlite.execute(sql)
rp.rowcount

6

In [66]:
#To force orders.csv and orders table to be same copy
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)
orders[cols].to_csv(one_file, header=True, index=False)

### End of Evening Process